In [1]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import functools
import pickle
from operator import add
import matplotlib as mpl
from alpdesign.utils import *
from alpdesign.mlp import *
#from jax_unirep import get_reps
import alpdesign
import os


In [2]:
AA_list = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V','B','Z','X','*']
blosum92 = np.loadtxt("./blosum62.txt", dtype='i', delimiter=' ')

avg92 = jnp.sum(blosum92)/24/24
sum92 = 0.
for row in blosum92:
    for aa in row:
        sum92 += (aa-avg92)**2
std92 = jnp.sqrt(sum92 / 24/24)

def blosum(seq1, seq2):
    seqlist1 = list(seq1)
    seqlist2 = list(seq2)
    score = 0.
    for i in range(len(seqlist1)):
        idx1 = AA_list.index(seqlist1[i])
        idx2 = AA_list.index(seqlist2[i])
        score += (blosum92[idx1][idx2] - avg92)/std92
        #jax.nn.sigmoid(score/len(seqlist1))
    return score/len(seqlist1)

In [3]:
target_seq = 'TARGETPEPTIDE'
key = jax.random.PRNGKey(0)
c = alpdesign.EnsembleBlockConfig()
forward_t, full_forward_t, seq_t , uncertainty_eval_t = alpdesign.build_e2e(c)
#def gen(k, n): return jax.random.normal(k, shape=(n, 13, 20))
train_t = full_forward_t
aconfig = AlgConfig()
c = alpdesign.EnsembleBlockConfig()

In [4]:
with open('../10kseqs.txt') as f:
    readfile = f.readlines()
    random_seqs = f'{readfile[0]}'.split(' ')[:-1]
    

def get_labels(seqs):
    labels = []
    for seq in seqs:
        labels.append(blosum(target_seq, seq))
    labels = np.array(labels)

    return labels

def get_aanum(seqs):
    aa_count = []
    for seq in seqs:
        seq_list = list(seq)
        aa_num = [float(seq_list.count(aa)) for aa in AA_list]
        aa_count.append(aa_num)
    aa_count = jnp.array(aa_count)
    return aa_count

batch_size = 8
def get_results(key, params, rep_list):
    means = []
    stds = []
    # need batch
    for i in range(0, len(rep_list)//batch_size):
    #for rep in rep_list:
        batch_reps = rep_list[i * batch_size: (i + 1) * batch_size]
        yhat = forward_t.apply(params, key, batch_reps)
        #print(yhat.shape)
        means.append(yhat[0])
        stds.append(yhat[1])
    return np.array(means), np.array(stds)

def get_single_results(key, params, rep_list):
    yhats = []
    for i in range(0, len(rep_list)//batch_size):
    # need batch
    #for rep in rep_list:
        batch_reps = rep_list[i*batch_size: (i+1)*batch_size]
        yhat = naive_forward_t.apply(params, key, batch_reps)
        yhats.append(yhat)
    return np.array(yhats)

In [5]:
validation_seqs = [random.choice(random_seqs) for i in range(50)]
validation_reps = get_aanum(validation_seqs)
validation_labels = get_labels(validation_seqs).reshape((-1, 1))
test_seqs = [random.choice(random_seqs) for i in range(500)]
test_reps = get_aanum(test_seqs)
test_labels = get_labels(test_seqs).reshape((-1, 1))
train_seqs = [random.choice(random_seqs) for i in range(100)]
train_reps = get_aanum(train_seqs)
train_labels = get_labels(train_seqs)

In [6]:
dummy_x = jnp.ones([c.model_number, aconfig.train_batch_size, 24])
params = train_t.init(key, dummy_x)
a = train_t.apply(params, key, dummy_x)
a0 = a[...,0]
a1 = a[...,1]
n_log_likelihoods = 0.5 * jnp.log(a1)
print(n_log_likelihoods.shape)
print(a.shape)
print(a0.shape)
loss = jnp.sum(n_log_likelihoods)
#print(loss.shape)

(5, 8)
(5, 8, 2)
(5, 8)


In [7]:
seq = dummy_x[0]
M = 5
print(seq.shape)
jnp.tile(seq[jnp.newaxis], (M,1,1)).shape

(8, 24)


(5, 8, 24)

In [9]:
aconfig.train_epochs=1

In [ ]:
params, train_loss, val_loss = ensemble_train(
        key, train_t, c, train_reps, train_labels, val_seqs=validation_reps, val_labels=validation_labels, aconfig=aconfig
    )

training in progress


2021-12-09 14:00:25.720018: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:55] 
********************************
Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
Compiling module jit_train_step.17716
********************************


In [ ]:
# test shape
batch_seqs = jnp.ones([8, train_reps.shape[-1]])
params = alpdesign.mlp.naive_forward_t.init(key, batch_seqs)
out = alpdesign.mlp.naive_forward_t.apply(params, key, batch_seqs)
print(out.shape)

In [ ]:
def _naive_loss(forward, key, params, seq, label):
    yhat = forward(params, key, seq) #scalar
    return (label-yhat)**2
batch_labels = train_labels[:8]
naive_loss = _naive_loss(alpdesign.mlp.naive_forward_t.apply, key, params, batch_seqs, batch_labels)

In [ ]:
batch_labels = batch_labels.reshape((-1, 1))
naive_loss = _naive_loss(alpdesign.mlp.naive_forward_t.apply, key, params, batch_seqs, batch_labels)
print(batch_seqs.shape)
print(batch_labels.shape)
print(naive_loss.shape)

In [ ]:
params, train_loss, val_losses = naive_train(
        key, naive_forward_t, train_reps, train_labels, val_seqs=validation_reps, val_labels=validation_labels, params=None
    )

In [ ]:
plt.figure()
plt.plot(train_loss, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.title('Loss')
plt.legend()
plt.xlabel('epochs')



In [ ]:

y_pred, y_std = get_results(key, params, test_reps)
#(exp_proportions, obs_proportions) = uct.get_proportion_lists(y_pred, y_std, validation_labels)



In [ ]:
y_pred = get_single_results(key, params, test_reps)

In [ ]:
def parity_plot(y_pred, test_labels):
    ground_truth = test_labels
    pred = y_pred

    plt.scatter(ground_truth, pred, c='lightskyblue', alpha=0.3)
    plt.plot( [-0.5,1.0],[-0.5,1.0], color='lightsalmon', linewidth=2)
    plt.title('Parity Plot')
    plt.xlim([-0.5, 1.0])
    plt.ylim([-0.5, 1.0])
    plt.xlabel('Ground truth')
    plt.ylabel('Predictions')
    plt.grid()
    return plt


def calibration_plot(y_pred, y_std, test_labels):
    (exp_proportions, obs_proportions) = uct.get_proportion_lists_vectorized(y_pred, y_std, test_labels)
    plt.figure()
    ax = plt.gca()
    ax.plot(exp_proportions, obs_proportions, c="#1f77b4")
    #ax.set_title('Calibration Parity Plot %d' % num)
    ax.set_title('Calibration Plot')
    ax.set_xlabel("Predicted Proportion in Interval")
    ax.set_ylabel("Observed Proportion in Interval")
    ax.axis("square")
    ax.grid()
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.0])
    ax.plot(exp_proportions, exp_proportions, '--', c='orange')
    ax.fill_between(exp_proportions, exp_proportions, obs_proportions, alpha=0.2)
    polygon_points = []
    for point in zip(exp_proportions, obs_proportions):
        polygon_points.append(point)
    for point in zip(reversed(exp_proportions), reversed(exp_proportions)):
        polygon_points.append(point)
    polygon_points.append((exp_proportions[0], obs_proportions[0]))
    polygon = Polygon(polygon_points)
    x, y = polygon.exterior.xy  # original data
    ls = LineString(np.c_[x, y])  # closed, non-simple
    lr = LineString(ls.coords[:] + ls.coords[0:1])
    mls = unary_union(lr)
    polygon_area_list = [poly.area for poly in polygonize(mls)]
    miscalibration_area = np.asarray(polygon_area_list).sum()
    ax.text(
            x=0.95,
            y=0.05,
            s="Miscalibration area = %.2f" % miscalibration_area,
            verticalalignment="bottom",
            horizontalalignment="right",
            fontsize="small",
        )

In [ ]:
print(y_pred.shape)
print(test_labels.shape)

In [ ]:
y_pred = y_pred.reshape((-1, 1))
parity_plot(y_pred, test_labels[:len(y_pred)])